# Analyzing Supreme Court decisions

Background: This notebook analyzes scraped information from the Philippine Supreme Court [website](https://elibrary.judiciary.gov.ph/).

## Do your imports

In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import re

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Read CSV

The file contains the data getting scraped from the Supreme Court website every Sunday.

In [2]:
df= pd.read_csv('sc-cases.csv')
df

,case_no,title,date,case_link,division,ponente
0,G.R. No. 205817,"REPUBLIC OF THE PHILIPPINES, PETITIONER, VS. L...","October 06, 2021",https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,"HERNANDO, J.:"
1,G.R. No. 202392,"PACIFIC ROYAL BASIC FOODS, INC., PETITIONER, V...","October 04, 2021",https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,"HERNANDO, J.:"
2,G.R. Nos. 199729-30,"MANILA BANKERS' LIFE INSURANCE CORPORATION, PE...","February 27, 2019",https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,"A. REYES, JR., J.:"
3,G.R. No. 229823,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...","February 27, 2019",https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,"DEL CASTILLO, J.:[*]"
4,G.R. No. 229099,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...","February 27, 2019",https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,"DEL CASTILLO, J.:[*]"
...,...,...,...,...,...,...
29711,G.R. No. 122807,"ROGELIO P. MENDIOLA, PETITIONER, VS. COURT OF ...","July 05, 1996",https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,"HERMOSISIMA, JR., J.:"
29712,G.R. No. 113549,"REPUBLIC OF THE PHILIPPINES, (REPRESENTED BY T...","July 05, 1996",https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,"DAVIDE JR., J.:"
29713,G.R. No. 116600,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...","July 03, 1996",https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,"DAVIDE, JR., J.:"
29714,G.R. No. 119527,"EVELYN J. GARCIA, PETITIONER, VS. NATIONAL LAB...","July 03, 1996",https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,"ROMERO, J.:"


## Cleaning the data

In [3]:
df.ponente.value_counts().sort_index()

A. REYES, JR., J.:           50
A. REYES, JR., J.:[*]         1
ABAD, J.:                   230
AUSTRIA-MARTINEZ,\r\nJ.:      8
AUSTRIA-MARTINEZ, J.         21
                           ... 
YNARES_SANTIAGO,\r\nJ.:       2
YNARES_SANTIAGO, J.:          4
ZALAMEDA, J.:                46
ZALAMEDA, R.V., J.:           2
Â MENDOZA, J.:                1
Name: ponente, Length: 297, dtype: int64

### Removing some elements from ponente name

Across different cases, names of justices who penned the decision are written differently. Cleaning them to ensure they are of the same format will allow us to get a clearer sense of how many decisions they have written once we **groupby**.

In [4]:
df.ponente = df.ponente.str.replace('J', '')
df.ponente = df.ponente.str.replace(':', "")
df.ponente = df.ponente.str.replace(',', '')
df.ponente = df.ponente.str.replace('.', '')
df.ponente = df.ponente.str.replace('*', '')
df.ponente = df.ponente.str.replace('[', '')
df.ponente = df.ponente.str.replace(']', '')
df.ponente = df.ponente.str.replace(';', '')
df.ponente = df.ponente.str.replace('\r', '')
df.ponente = df.ponente.str.replace('\n', '')

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_3636/3550154347.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_3636/3550154347.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_3636/3550154347.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_3636/3550154347.py:7: FutureWarning: The default value of regex will change from True to Fal

In [5]:
df.ponente = df.ponente.str.replace('Acting C', '')
df.ponente = df.ponente.str.replace('ACTING C', '')
df.ponente.str.strip() 

0             HERNANDO
1             HERNANDO
2            A REYES R
3         DEL CASTILLO
4         DEL CASTILLO
             ...      
29711    HERMOSISIMA R
29712         DAVIDE R
29713         DAVIDE R
29714           ROMERO
29715             MELO
Name: ponente, Length: 29716, dtype: object

In [6]:
df.ponente.value_counts().sort_index()

 REYES R                   71
 STEVENS-LERK OF COURT      1
A REYES R                  51
ABAD                      230
ARDELEZA                  169
                         ... 
YNARES_SANTIAGO             2
YNARES_SANTIAGO             4
ZALAMEDA                   46
ZALAMEDA RV                 2
Â MENDOZA                   1
Name: ponente, Length: 175, dtype: int64

## Convert date to datetime

This will allow us to easily analyze decisions by timeframe. For instance, knowing how many decisions are promulgated at certain period of time.

In [7]:
df.date = pd.to_datetime(df.date)

In [8]:
df

,case_no,title,date,case_link,division,ponente
0,G.R. No. 205817,"REPUBLIC OF THE PHILIPPINES, PETITIONER, VS. L...",2021-10-06,https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,HERNANDO
1,G.R. No. 202392,"PACIFIC ROYAL BASIC FOODS, INC., PETITIONER, V...",2021-10-04,https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,HERNANDO
2,G.R. Nos. 199729-30,"MANILA BANKERS' LIFE INSURANCE CORPORATION, PE...",2019-02-27,https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,A REYES R
3,G.R. No. 229823,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...",2019-02-27,https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,DEL CASTILLO
4,G.R. No. 229099,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...",2019-02-27,https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,DEL CASTILLO
...,...,...,...,...,...,...
29711,G.R. No. 122807,"ROGELIO P. MENDIOLA, PETITIONER, VS. COURT OF ...",1996-07-05,https://elibrary.judiciary.gov.ph/thebookshelf...,FIRST DIVISION,HERMOSISIMA R
29712,G.R. No. 113549,"REPUBLIC OF THE PHILIPPINES, (REPRESENTED BY T...",1996-07-05,https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,DAVIDE R
29713,G.R. No. 116600,"PEOPLE OF THE PHILIPPINES, PLAINTIFF-APPELLEE,...",1996-07-03,https://elibrary.judiciary.gov.ph/thebookshelf...,THIRD DIVISION,DAVIDE R
29714,G.R. No. 119527,"EVELYN J. GARCIA, PETITIONER, VS. NATIONAL LAB...",1996-07-03,https://elibrary.judiciary.gov.ph/thebookshelf...,SECOND DIVISION,ROMERO


## Initial analysis